# Reading all the traces from Set 1 and Set 2

Descriptions of the traces:

*** Naming convention:

Each file corresponds to an execution trace of a multi-mode system under a specific protocol, where the system moves through a series of modes during the experiment interval. The trace shows the timeline of tasks that are scheduled on the CPU. All tasks are implicit-deadline periodic tasks, and the scheduler is EDF. 

The file name indicates the number of tasks (across all modes) and the protocol used. For example, 

v4-async_no_period-trace.txt: the multi-mode system has four tasks, and the protocol Asynchronous without Periodicity is used.

*** Trace format:

Each file has four columns:

- Column 1: The current value of the CPU tick (in decimal value) -- the CPU ran at 3.4GHz. 

- Column 2: The ID of the task that is scheduled on the CPU at the corresponding CPU tick. 
  There is a special ID called "_IDLE_VCPU", which indicates the time at which the system is idle.

- Column 3: The absolute deadline of the task being scheduled, in hexadecimal and nanoseconds.

- Column 4: The remaining execution time of the task being scheduled, in hexadecimal and nanoseconds.


Example: consider the file v4-async_no_period-trace.txt and its first two lines:

1774634982816621 0x00350000 0x0001dbca884e4275 0x0000000000249f00
1774634990961622 0x00350001 0x0001dbca889a94d3 0x000000000036ee80
...

The files shows a trace of a system has 4 tasks (with ID in 0x00350000-0x00350003), and the protocol asynchronous without periodicity is used. The two lines of the trace indicate that 
- At tick 1774634982816621, task 0x00350000 is scheduled, and its deadline and remaining WCET are 0x0001dbca884e4275 and 0x0000000000249f00, respectively.
- At tick 1774634990961622, task 0x00350001 is then scheduled, and its deadline and remaining WCET are 0x0001dbca889a94d3 and 0x000000000036ee80, respectively.

*** Important notes:

1) As in any real system, there are various sources of overheads; hence, the data are not always exactly the same as the values expected theoretically. For instance, 

- The observed execution time of a task varies slightly across jobs and is NOT always exactly the same as the task's WCET. This is because of the overhead value (in the scheduling function, timer overhead, context switches, etc) varies over time. 

- Suppose a job is released at tick t and it is the highest priority among all pending jobs. In theory, we expect that it will be scheduled at tick t; however, in practice, it will be delayed because of the overhead.

- The tracing system traces the scheduling events in tick values, whereas the task parameters (e.g., timer) are in nanoseconds. Hence, you should expect some noise due to the conversion. 

- ...

Hence, I think we need to be able to consider such practical issues in deriving the reverse engineering solution.

2) The deadlines and remaining WCETs (Column 3+4) are provided in the trace so that you can check whether your reverse engineering does give the correct timing parameters; I suppose you should not rely on them in deriving the task parameters.

For your information, each system has four modes, and the mode switches happen in a cyclic fashion, i.e., Mode 0 --> Mode 1 --> Mode 2 --> Mode 3 --> Mode 0 --> ...



# Import all the necessary libraries 

In [1]:
import numpy as np
import pandas as pd

# Reading raw trace from the given file

In [2]:
def readTraceFromAFile(filepath):
    traceArray = []
    with open(filepath) as fp:  
        line = fp.readline()
        cnt = 1
        while line:
            traceArray.append(line)
            #print("Line {}: {}".format(cnt, line.strip()))
            line = fp.readline()
            cnt += 1
    return traceArray

# Formatting the trace array 
Converting hexadecimal to decimal and polishing the values.

In [3]:
def formatTheTraceArray(traceArray):
    currentline = [4]
    formattedTraceArray = []
    i = 0
    for line in traceArray:
        formattedTraceArray.append([])
        currentline = line.split();
        formattedTime= int(currentline[0])
        formattedTraceArray[i].append(formattedTime)
        
        if currentline[1] == "_IDLE_VCPU":
            formattedId= 0
        else:
            formattedId= int(currentline[1],0)
            
        formattedTraceArray[i].append(formattedId)
        
        formattedDeadline= int(currentline[2],0)
        formattedTraceArray[i].append(formattedDeadline)
        
        formattedRemainingWCET= int(currentline[3],0)
        formattedTraceArray[i].append(formattedRemainingWCET) 
        
        i=i+1
    return formattedTraceArray

# Converting the trace array to Dataframe for better manipulation

In [4]:
def convertTraceToDataframe(formattedTraceArray):
    # convert the trace array to dataframe
    formattedTraceDataframe = pd.DataFrame({'CPU_Tick':np.array(formattedTraceArray)[:,0],
                                            'Task_Id':np.array(formattedTraceArray)[:,1],
                                            'Abs_Deadline':np.array(formattedTraceArray)[:,2],
                                            'Rem_WCET':np.array(formattedTraceArray)[:,3],
                                            'Rel_Deadline':np.zeros(np.size(formattedTraceArray,0)),
                                            'Ch_Rem_WCET':np.zeros(np.size(formattedTraceArray,0)),
                                            'Arrival_timeDiff':np.zeros(np.size(formattedTraceArray,0)),
                                            'IsDeadlineChanged':np.zeros(np.size(formattedTraceArray,0)),
                                            'IsWCETChanged':np.zeros(np.size(formattedTraceArray,0)),
                                            'IsArrivalTimeChanged':np.zeros(np.size(formattedTraceArray,0)),
                                            'IsTaskPropertyChanged':np.zeros(np.size(formattedTraceArray,0)),
                                            'IsModeChanged1':np.zeros(np.size(formattedTraceArray,0)),
                                            'IsModeChanged2':np.zeros(np.size(formattedTraceArray,0))},
                                            columns=['CPU_Tick','Task_Id','Abs_Deadline','Rem_WCET',
                                                    'Rel_Deadline','Ch_Rem_WCET','Arrival_timeDiff',
                                                    'IsDeadlineChanged','IsWCETChanged','IsArrivalTimeChanged',
                                                    'IsTaskPropertyChanged','IsModeChanged1','IsModeChanged2'])
    return formattedTraceDataframe

# Determine the number of tasks and id of each task in the trace

In [5]:
def identifyTasksFromTrace(formattedTraceDataframe):
    taskInfo = formattedtraceDataframe['Task_Id'].unique()
    #print(taskInfo)
    return taskInfo

# Extract new features and add them to the Dataframe

In [6]:
def extractFeaturesFromTrace(tasks,formattedTraceDataframe):
    for t in np.nditer(tasks):
        if t == 0:
            continue
        count = 0
        for index, row in formattedTraceDataframe.iterrows():
            if row['Task_Id'] == t:
                current = row
                if count == 0:
                    previous = row
                else:
                    row['Rel_Deadline'] = abs(current['Abs_Deadline'] - previous['Abs_Deadline'])
                    row['Ch_Rem_WCET'] = abs(current['Rem_WCET'] - previous['Rem_WCET'])
                    row['Arrival_timeDiff'] = abs(current['CPU_Tick'] - previous['CPU_Tick'])
                    
                    if previous['Rel_Deadline'] == 0 or current['Rel_Deadline'] == 0:
                        row['IsDeadlineChanged'] = 0
                    else:
                        row['IsDeadlineChanged'] = abs(current['Rel_Deadline'] - previous['Rel_Deadline'])
                        
                    if row['IsDeadlineChanged'] > 0:
                        row['IsTaskPropertyChanged'] = 1
                    
                    if previous['Ch_Rem_WCET'] == 0 or current['Ch_Rem_WCET'] == 0:
                        row['IsWCETChanged'] = 0
                    else:
                        row['IsWCETChanged'] = abs(current['Ch_Rem_WCET'] - previous['Ch_Rem_WCET'])
                        
                        
                    row['IsArrivalTimeChanged'] = abs(current['Arrival_timeDiff'] - previous['Arrival_timeDiff'])

                    formattedTraceDataframe.at[index,'Rel_Deadline'] = row['Rel_Deadline']
                    formattedTraceDataframe.at[index,'Ch_Rem_WCET'] = row['Ch_Rem_WCET']
                    formattedTraceDataframe.at[index,'Arrival_timeDiff'] = row['Arrival_timeDiff']
                    formattedTraceDataframe.at[index,'IsDeadlineChanged'] = row['IsDeadlineChanged']
                    formattedTraceDataframe.at[index,'IsWCETChanged'] = row['IsWCETChanged']
                    formattedTraceDataframe.at[index,'IsArrivalTimeChanged'] = row['IsArrivalTimeChanged']
                    formattedTraceDataframe.at[index,'IsTaskPropertyChanged'] = row['IsTaskPropertyChanged']

                    previous = row
                count = count + 1
                
    return formattedTraceDataframe
    #print(tasks)
    #print(formattedTraceDataframe)

# Identification of possible mode changes based on the properties of the tasks
Here, we aim to identify and categorize the execution traces based on the properties of the tasks

In [7]:
def ClusteringBasedOnTaskProperties(formattedTraceDataframe): 
    flaggedTask = 0;
    for index, row in formattedTraceDataframe.iterrows():
            if row['IsTaskPropertyChanged'] == 1:
                if flaggedTask == 0:
                    formattedTraceDataframe.at[index,'IsModeChanged1'] = 1
                    flaggedTask = row['Task_Id']
                    flaggedIndex = index
                    flaggedRemWCET = row['Rem_WCET']
                    
                if row['Task_Id'] == flaggedTask:
                    if row['Rem_WCET'] != flaggedRemWCET: 
                        formattedTraceDataframe.at[index,'IsModeChanged1'] = 1
                    flaggedTask = row['Task_Id']
                    flaggedIndex = index
                    flaggedRemWCET = row['Rem_WCET'] 
                    
                if row['Task_Id'] != flaggedTask and (flaggedIndex + 10) >= index:
                    flaggedTask = row['Task_Id']
                    flaggedIndex = index
                    flaggedRemWCET = row['Rem_WCET']
                    
                if row['Task_Id'] != flaggedTask and (flaggedIndex + 10) < index:
                    formattedTraceDataframe.at[index,'IsModeChanged1'] = 1
                    flaggedTask = row['Task_Id']
                    flaggedIndex = index
                    flaggedRemWCET = row['Rem_WCET']
                    
    return formattedTraceDataframe

# Working with v4-async_no_period-trace

# Reading v4-async_no_period-trace

In [8]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v4-async_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v4-async_no_period-trace and Converting v4-async_no_period-trace to Dataframe

In [9]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [10]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [11]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v4-async_no_period-trace.csv')

# Working with v4-async_period-trace

# Reading v4-async_period-trace

In [12]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v4-async_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v4-async_period-trace and Converting v4-async_period-trace to Dataframe

In [13]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [14]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [15]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v4-async_period-trace.csv')

# Working with v4-max_period_offset-trace

# Reading v4-max_period_offset-trace

In [16]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v4-max_period_offset-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v4-max_period_offset-trace and Converting v4-max_period_offset-trace to Dataframe

In [17]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [18]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [19]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v4-max_period_offset-trace.csv')

# Working with v4-min_offset_no_period-trace

# Reading v4-min_offset_no_period-trace

In [20]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v4-min_offset_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v4-min_offset_no_period-trace and Converting v4-min_offset_no_period-trace to Dataframe

In [21]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [22]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [23]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v4-min_offset_no_period-trace.csv')

# Working with v4-min_offset_with_peri-trace

# Reading v4-min_offset_with_peri-trace

In [24]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v4-min_offset_with_peri-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v4-min_offset_with_peri-trace and Converting v4-min_offset_with_peri-trace to Dataframe

In [25]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [26]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [27]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v4-min_offset_with_peri-trace.csv')

# Working with v8-async_no_period-trace

# Reading v8-async_no_period-trace

In [28]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v8-async_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v8-async_no_period-trace and Converting v8-async_no_period-trace to Dataframe

In [29]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [30]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [31]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v8-async_no_period-trace.csv')

# Working with v8-async_period-trace

# Reading v8-async_period-trace

In [32]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v8-async_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v8-async_period-trace and Converting v8-async_period-trace to Dataframe

In [33]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [34]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [35]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v8-async_period-trace.csv')

# Working with v8-max_period_offset-trace

# Reading v8-max_period_offset-trace

In [36]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v8-max_period_offset-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v8-max_period_offset-trace and Converting v8-max_period_offset-trace to Dataframe

In [37]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [38]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [39]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v8-max_period_offset-trace.csv')

# Working with v8-min_offset_no_period-trace

# Reading v8-min_offset_no_period-trace

In [40]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v8-min_offset_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v8-min_offset_no_period-trace and Converting v8-min_offset_no_period-trace to Dataframe

In [41]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [42]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [43]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v8-min_offset_no_period-trace.csv')

# Working with v8-min_offset_with_peri-trace

# Reading v8-min_offset_with_peri-trace

In [44]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v8-min_offset_with_peri-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v8-min_offset_with_peri-trace and Converting v8-min_offset_with_peri-trace to Dataframe

In [45]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [46]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [47]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v8-min_offset_with_peri-trace.csv')

# Working with v16-async_no_period-trace

# Reading v16-async_no_period-trace

In [48]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v16-async_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v16-async_no_period-trace and Converting v16-async_no_period-trace to Dataframe

In [49]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [50]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [51]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v16-async_no_period-trace.csv')

# Working with v16-async_period-trace

# Reading v16-async_period-trace

In [52]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v16-async_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v16-async_period-trace and Converting v16-async_period-trace to Dataframe

In [53]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [54]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [55]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v16-async_period-trace.csv')

# Working with v16-max_period_offset-trace

# Reading v16-max_period_offset-trace

In [56]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v16-max_period_offset-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v16-max_period_offset-trace and Converting v16-max_period_offset-trace to Dataframe

In [57]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [58]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [59]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v16-max_period_offset-trace.csv')

# Working with v16-min_offset_no_period-trace

# Reading v16-min_offset_no_period-trace

In [60]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v4-async_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v16-min_offset_no_period-trace and Converting v16-min_offset_no_period-trace to Dataframe

In [61]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [62]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [63]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v16-min_offset_no_period-trace.csv')

# Working with v16-min_offset_with_peri-trace

# Reading v16-min_offset_with_peri-trace

In [64]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v16-min_offset_with_peri-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v16-min_offset_with_peri-trace and Converting v16-min_offset_with_peri-trace to Dataframe

In [65]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [66]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [67]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v16-min_offset_with_peri-trace.csv')

# Working with v32-async_no_period-trace

# Reading v32-async_no_period-trace

In [68]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v32-async_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v4-async_no_period-trace and Converting v4-async_no_period-trace to Dataframe

In [69]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [70]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [71]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v32-async_no_period-trace.csv')

# Working with v32-async_period-trace

# Reading v32-async_period-trace

In [72]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v32-async_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v32-async_period-trace and Converting v32-async_period-trace to Dataframe

In [73]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [74]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [75]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v32-async_period-trace.csv')

# Working with v32-max_period_offset-trace

# Reading v32-max_period_offset-trace

In [76]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v32-max_period_offset-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v32-max_period_offset-trace and Converting v32-max_period_offset-trace to Dataframe

In [77]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [78]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [79]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v32-max_period_offset-trace.csv')

# Working with v32-min_offset_no_period-trace

# Reading v32-min_offset_no_period-trace

In [80]:
v4asyncnoperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v32-min_offset_no_period-trace.txt' 
v4asyncnoperiodtrace = readTraceFromAFile(filepath)

# Formatting v32-min_offset_no_period-trace and Converting v32-min_offset_no_period-trace to Dataframe

In [81]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe

,CPU_Tick,Task_Id,Abs_Deadline,Rem_WCET,Rel_Deadline,Ch_Rem_WCET,Arrival_timeDiff,IsDeadlineChanged,IsWCETChanged,IsArrivalTimeChanged,IsTaskPropertyChanged,IsModeChanged1,IsModeChanged2
0,1785791211288041,4718592,526426735551709,300000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1785791212308313,4718593,526426736556811,330000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1785791213438490,4718594,526426737558495,360000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1785791214668151,4718595,526426738560194,390000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1785791216001342,4718596,526426739561857,420000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1785791217434187,4718597,526426740563491,450000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1785791218968590,4718598,526426741565105,480000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1785791220606812,4718599,526426742566748,510000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1785791222346847,4718600,526426743568332,540000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1785791224186077,4718601,526426744570101,570000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [82]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [83]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v32-min_offset_no_period-trace.csv')

# Working with v32-min_offset_with_peri-trace

# Reading v32-min_offset_with_peri-trace

In [84]:
v4asyncperiodtrace = []
filepath = r'C:\Users\100631155\Google Drive\WaterLoo Meeting\safeMC_simple_traces\safeMC_simple_traces\SET1\v32-min_offset_with_peri-trace.txt' 
v4asyncperiodtrace = readTraceFromAFile(filepath)

# Formatting v32-min_offset_with_peri-trace and Converting v32-min_offset_with_peri-trace to Dataframe

In [85]:
formattedv4asyncnoperiodtrace = formatTheTraceArray(v4asyncnoperiodtrace)
formattedv4asyncnoperiodDataframe = convertTraceToDataframe(formattedv4asyncnoperiodtrace)
#formattedv4asyncnoperiodDataframe

# Identifying number and id the tasks as well as Extracting unique features from the formatted dataframe

In [86]:
taskInfo = formattedv4asyncnoperiodDataframe['Task_Id'].unique()
featuredDataframe = extractFeaturesFromTrace(taskInfo,formattedv4asyncnoperiodDataframe)

# Performing initial clustering based on task properties and Writing the data to CSV
This involves identifying the time ticks from where properties of (some of) the tasks have started changing. 
Note: Minor changes were ignored

In [87]:
v4asyncnoperiodDataframe = ClusteringBasedOnTaskProperties(featuredDataframe)
v4asyncnoperiodDataframe.to_csv(r'C:\Users\100631155\Desktop\TraceProject\v32-min_offset_with_peri-trace.csv')

In [89]:
v4asyncnoperiodDataframe

,CPU_Tick,Task_Id,Abs_Deadline,Rem_WCET,Rel_Deadline,Ch_Rem_WCET,Arrival_timeDiff,IsDeadlineChanged,IsWCETChanged,IsArrivalTimeChanged,IsTaskPropertyChanged,IsModeChanged1,IsModeChanged2
0,1785791211288041,4718592,526426735551709,300000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
1,1785791212308313,4718593,526426736556811,330000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
2,1785791213438490,4718594,526426737558495,360000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
3,1785791214668151,4718595,526426738560194,390000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
4,1785791216001342,4718596,526426739561857,420000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
5,1785791217434187,4718597,526426740563491,450000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
6,1785791218968590,4718598,526426741565105,480000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
7,1785791220606812,4718599,526426742566748,510000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
8,1785791222346847,4718600,526426743568332,540000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
9,1785791224186077,4718601,526426744570101,570000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0


# Extract more features for Mode identification and arrival of MCR

In [88]:
task1Flag = 0
task2Flag = 0
task3Flag = 0
task4Flag = 0

currentT1Deadline = 0
currentT1WCET = 0
changedT1Deadline = 0
changedT1WCET = 0

currentT2Deadline = 0
currentT2WCET = 0
changedT2Deadline = 0
changedT2WCET = 0

currentT3Deadline = 0
currentT3WCET = 0
changedT3Deadline = 0
changedT3WCET = 0

currentT4Deadline = 0
currentT4WCET = 0
changedT4Deadline = 0
changedT4WCET = 0

task1TimeDiff = 0
task2TimeDiff = 0
task3TimeDiff = 0
task4TimeDiff = 0
        
updatedv4asyncnoperiodtrace = []

for i in range(17973):
    updatedv4asyncnoperiodtrace.append([])
    if formattedv4asyncnoperiodtrace[i][1] == 3473408:
        taskName = "Task 1"
        if(task1Flag == 0):
            currentT1Deadline = formattedv4asyncnoperiodtrace[i][2]
            currentT1WCET = formattedv4asyncnoperiodtrace[i][3]
            task1Flag = 1
        else:
            changedT1Deadline = currentT1Deadline - formattedv4asyncnoperiodtrace[i][2]
            changedT1WCET = currentT1WCET - formattedv4asyncnoperiodtrace[i][3]
            task1TimeDiff = formattedv4asyncnoperiodtrace[i][0] - prevT1ArrivalTime
            if (changedT1Deadline != 0 or changedT1WCET != 0):
                currentT1Deadline = formattedv4asyncnoperiodtrace[i][2]
                currentT1WCET = formattedv4asyncnoperiodtrace[i][3]
                
        currentT1ExecutionTime = formattedv4asyncnoperiodtrace[i+1][0] - formattedv4asyncnoperiodtrace[i][0]
        prevT1ArrivalTime = formattedv4asyncnoperiodtrace[i][0]
        
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][0])
        updatedv4asyncnoperiodtrace[i].append(taskName)
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][1])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][2])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][3])
        updatedv4asyncnoperiodtrace[i].append(changedT1Deadline)
        updatedv4asyncnoperiodtrace[i].append(changedT1WCET)
        updatedv4asyncnoperiodtrace[i].append(currentT1ExecutionTime)
        updatedv4asyncnoperiodtrace[i].append(task1TimeDiff)
        
        #print(formattedv4asyncnoperiodtrace[i][0],formattedv4asyncnoperiodtrace[i][1],
        #      formattedv4asyncnoperiodtrace[i][2],formattedv4asyncnoperiodtrace[i][3],
        #      taskName,changedT1Deadline,changedT1WCET,currentT1ExecutionTime,task1TimeDiff)
    
    if formattedv4asyncnoperiodtrace[i][1] == 3473409:
        taskName = "Task 2"
        if(task2Flag == 0):
            currentT2Deadline = formattedv4asyncnoperiodtrace[i][2]
            currentT2WCET = formattedv4asyncnoperiodtrace[i][3]
            task2Flag = 1
        else:
            changedT2Deadline = currentT2Deadline - formattedv4asyncnoperiodtrace[i][2]
            changedT2WCET = currentT2WCET - formattedv4asyncnoperiodtrace[i][3]
            task2TimeDiff = formattedv4asyncnoperiodtrace[i][0] - prevT2ArrivalTime
            if (changedT2Deadline != 0 or changedT2WCET != 0):
                currentT2Deadline = formattedv4asyncnoperiodtrace[i][2]
                currentT2WCET = formattedv4asyncnoperiodtrace[i][3]
        currentT2ExecutionTime = formattedv4asyncnoperiodtrace[i+1][0] - formattedv4asyncnoperiodtrace[i][0]
        prevT2ArrivalTime = formattedv4asyncnoperiodtrace[i][0]
        
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][0])
        updatedv4asyncnoperiodtrace[i].append(taskName)
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][1])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][2])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][3])
        updatedv4asyncnoperiodtrace[i].append(changedT2Deadline)
        updatedv4asyncnoperiodtrace[i].append(changedT2WCET)
        updatedv4asyncnoperiodtrace[i].append(currentT2ExecutionTime)
        updatedv4asyncnoperiodtrace[i].append(task2TimeDiff)
        
        #print(formattedv4asyncnoperiodtrace[i][0],formattedv4asyncnoperiodtrace[i][1],formattedv4asyncnoperiodtrace[i][2],formattedv4asyncnoperiodtrace[i][3],taskName,changedT2Deadline,changedT2WCET,currentT2ExecutionTime,task2TimeDiff)
    
    if formattedv4asyncnoperiodtrace[i][1] == 3473410:
        taskName = "Task 3"
        if(task3Flag == 0):
            currentT3Deadline = formattedv4asyncnoperiodtrace[i][2]
            currentT3WCET = formattedv4asyncnoperiodtrace[i][3]
            task3Flag = 1
        else:
            changedT3Deadline = currentT3Deadline - formattedv4asyncnoperiodtrace[i][2]
            changedT3WCET = currentT3WCET - formattedv4asyncnoperiodtrace[i][3]
            task3TimeDiff = formattedv4asyncnoperiodtrace[i][0] - prevT3ArrivalTime
            if (changedT3Deadline != 0 or changedT3WCET != 0):
                currentT3Deadline = formattedv4asyncnoperiodtrace[i][2]
                currentT3WCET = formattedv4asyncnoperiodtrace[i][3]
        currentT3ExecutionTime = formattedv4asyncnoperiodtrace[i+1][0] - formattedv4asyncnoperiodtrace[i][0]
        prevT3ArrivalTime = formattedv4asyncnoperiodtrace[i][0]
        
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][0])
        updatedv4asyncnoperiodtrace[i].append(taskName)
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][1])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][2])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][3])
        updatedv4asyncnoperiodtrace[i].append(changedT3Deadline)
        updatedv4asyncnoperiodtrace[i].append(changedT3WCET)
        updatedv4asyncnoperiodtrace[i].append(currentT3ExecutionTime)
        updatedv4asyncnoperiodtrace[i].append(task3TimeDiff)
        
        #print(formattedv4asyncnoperiodtrace[i][0],formattedv4asyncnoperiodtrace[i][1],formattedv4asyncnoperiodtrace[i][2],formattedv4asyncnoperiodtrace[i][3],taskName,changedT3Deadline,changedT3WCET,currentT3ExecutionTime,task3TimeDiff)
    
    if formattedv4asyncnoperiodtrace[i][1] == 3473411:
        taskName = "Task 4"
        if(task4Flag == 0):
            currentT4Deadline = formattedv4asyncnoperiodtrace[i][2]
            currentT4WCET = formattedv4asyncnoperiodtrace[i][3]
            task4Flag = 1
        else:
            changedT4Deadline = currentT4Deadline - formattedv4asyncnoperiodtrace[i][2]
            changedT4WCET = currentT4WCET - formattedv4asyncnoperiodtrace[i][3]
            task4TimeDiff = formattedv4asyncnoperiodtrace[i][0] - prevT4ArrivalTime
            if (changedT4Deadline != 0 or changedT4WCET != 0):
                currentT4Deadline = formattedv4asyncnoperiodtrace[i][2]
                currentT4WCET = formattedv4asyncnoperiodtrace[i][3]
        
        currentT4ExecutionTime = formattedv4asyncnoperiodtrace[i+1][0] - formattedv4asyncnoperiodtrace[i][0]
        prevT4ArrivalTime = formattedv4asyncnoperiodtrace[i][0]
        
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][0])
        updatedv4asyncnoperiodtrace[i].append(taskName)
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][1])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][2])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][3])
        updatedv4asyncnoperiodtrace[i].append(changedT4Deadline)
        updatedv4asyncnoperiodtrace[i].append(changedT4WCET)
        updatedv4asyncnoperiodtrace[i].append(currentT4ExecutionTime)
        updatedv4asyncnoperiodtrace[i].append(task4TimeDiff)
        
        #print(formattedv4asyncnoperiodtrace[i][0],formattedv4asyncnoperiodtrace[i][1],formattedv4asyncnoperiodtrace[i][2],formattedv4asyncnoperiodtrace[i][3],taskName,changedT4Deadline,changedT4WCET,currentT4ExecutionTime,task4TimeDiff)
    
    if formattedv4asyncnoperiodtrace[i][1] == 0:
        taskName = "No task"
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][0])
        updatedv4asyncnoperiodtrace[i].append(taskName)
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][1])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][2])
        updatedv4asyncnoperiodtrace[i].append(formattedv4asyncnoperiodtrace[i][3])
        updatedv4asyncnoperiodtrace[i].append(0)
        updatedv4asyncnoperiodtrace[i].append(0)
        updatedv4asyncnoperiodtrace[i].append(0)
        updatedv4asyncnoperiodtrace[i].append(0)
        
        #print(formattedv4asyncnoperiodtrace[i][0],formattedv4asyncnoperiodtrace[i][1],formattedv4asyncnoperiodtrace[i][2],formattedv4asyncnoperiodtrace[i][3],taskName)
    #print(updatedv4asyncnoperiodtrace[i])